# Projeto 2 - Ciência dos Dados

Nome: Luiz Felipe Domingues Valente

Nome: João Guilherme Cintra de Freitas Almeida

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [78]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [79]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import numpy as np
import re 

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [80]:
#Dados de autenticação do twitter:

# Coloque aqui o identificador da conta no twitter: @jofenina

#leitura do arquivo no formato JSON
# with open('auth.pass') as fp:    
#     data = json.load(fp)

#Configurando a biblioteca. Não modificar
# auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
# auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [81]:
#Produto escolhido:
produto = 'snickers'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [82]:
#Cria um objeto para a captura
# api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
# i = 1
# msgs = []
# for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
#     msgs.append(msg.full_text.lower())
#     i += 1
#     if i > n:
#         break

#Embaralhando as mensagens para reduzir um possível viés
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [83]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
# if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
#     writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
#     dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
#     dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

#     dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
#     dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
#     writer.save()

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

# Limpeza

In [84]:
tweets_tr_sujos = pd.read_excel("snickers.xlsx")
tweets_tr_sujos.head()

,Treinamento,Relevância
0,@tiagoatoc nunca pensei que me mandasses comer...,0
1,"credo q eu acordei sem paciência, e já to come...",0
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
4,@yenexiel ninguém compra snickers pra mim quan...,0


In [85]:
treinamento = tweets_tr_sujos.loc[:, "Treinamento"]
rel=tweets_tr_sujos.loc[:, "Relevância"]
treinamento.head()

0    @tiagoatoc nunca pensei que me mandasses comer...
1    credo q eu acordei sem paciência, e já to come...
2    samyra me deu batata frita com cheddar e bacon...
3    crlh acabei de descobrir q tem sorvete de snic...
4    @yenexiel ninguém compra snickers pra mim quan...
Name: Treinamento, dtype: object

In [86]:
def cleanup(text):
    
#  Limpeza de caracteres

    punctuation = '[!\-.:?;@]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [87]:
t_limpos1 = []
t_limpos2 = []
t_limpos = []
for linha in treinamento:
    b=linha.lower()
    t_limpos1.append(cleanup(b))
for e in t_limpos1:
    a = e.strip()
    t_limpos2.append(a)
for s in t_limpos2:
    z = ' '.join(s.split())
    t_limpos.append(z)
t_limpos[0:5]

['tiagoatoc nunca pensei que me mandasses comer snickers antes de dormir sinceramente',
 'credo q eu acordei sem paciência, e já to comendo um snickers de café da manhã',
 'samyra me deu batata frita com cheddar e bacon, e evelyn me deu snickers, amo minhas amigas',
 'crlh acabei de descobrir q tem sorvete de snickers eu preciso pra ontem',
 'yenexiel ninguém compra snickers pra mim quando tô triste']

# Tabela com limpeza e relevância

In [104]:
t_tr = pd.DataFrame({"Treinamento": t_limpos,"Relevância": rel})
t_tr.head()

,Treinamento,Relevância
0,tiagoatoc nunca pensei que me mandasses comer ...,0
1,"credo q eu acordei sem paciência, e já to come...",0
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
4,yenexiel ninguém compra snickers pra mim quand...,0


# Tweets Relevantes Limpos e frequência de suas Palavras

In [89]:
t_rel= t_tr[(t_tr.Relevância==1)]
t_rel.head()

,Treinamento,Relevância
2,samyra me deu batata frita com cheddar e bacon...,1
3,crlh acabei de descobrir q tem sorvete de snic...,1
6,"rt thaii_xx sério, o anúncio da snickers no yt...",1
7,rt amandinhavr eu esperando a propaganda insup...,1
8,"não aguento mais esses anúncios da snickers, n...",1


In [90]:
serie_tweets = ' '.join(t_rel.loc[:,"Treinamento"])
a = pd.Series(serie_tweets.split())
a.head()

0    samyra
1        me
2       deu
3    batata
4     frita
dtype: object

In [91]:
a.value_counts().head()

snickers    166
de           92
eu           69
do           62
e            55
dtype: int64

In [92]:
tabela_tweets_relativa = a.value_counts(True)
tabela_tweets_relativa.head()

snickers    0.066188
de          0.036683
eu          0.027512
do          0.024721
e           0.021930
dtype: float64

In [93]:
imprime = 4
for palavra in tabela_tweets_relativa.index:
    if imprime:
        print(tabela_tweets_relativa[palavra])
        imprime-=1

0.06618819776714513
0.03668261562998405
0.02751196172248804
0.024720893141945772


# Tweets não Relevantes Limpos e frequência de suas Palavras

In [110]:
t_n_rel= t_tr[(t_tr.Relevância==0)]
t_n_rel.head()

,Treinamento,Relevância
0,tiagoatoc nunca pensei que me mandasses comer ...,0
1,"credo q eu acordei sem paciência, e já to come...",0
4,yenexiel ninguém compra snickers pra mim quand...,0
5,se tu roubar uma birosca dentro da favela os c...,0
15,a casa do jovem surtado se resume a pacotes de...,0


In [95]:
serie_tweets_n_rel = ' '.join(t_n_rel.loc[:,"Treinamento"])
a_n_rel = pd.Series(serie_tweets_n_rel.split())
a.head()

0    samyra
1        me
2       deu
3    batata
4     frita
dtype: object

In [96]:
a_n_rel.value_counts().head()

snickers    101
e            64
um           54
de           51
o            40
dtype: int64

In [97]:
tabela_tweets_relativa_n_rel = a_n_rel.value_counts(True)
tabela_tweets_relativa_n_rel.head()

snickers    0.052089
e           0.033007
um          0.027849
de          0.026302
o           0.020629
dtype: float64

In [98]:
imprime = 4
for palavra in tabela_tweets_relativa_n_rel.index:
    if imprime:
        print(tabela_tweets_relativa_n_rel[palavra])
        imprime-=1

0.05208870551830841
0.03300670448684889
0.02784940691077875
0.02630221763795771


In [116]:
dic_log = {}
for frase in t_tr.Treinamento:
    frase_d=frase.split()
    freq_log = 0 
    for palavra in frase_d:
        if palavra in tabela_tweets_relativa:
            freq_log += np.log10(tabela_tweets_relativa[palavra])
    dic_log[frase] = freq_log
dic_log

{'tiagoatoc nunca pensei que me mandasses comer snickers antes de dormir sinceramente': -11.625032159823007,
 'credo q eu acordei sem paciência, e já to comendo um snickers de café da manhã': -22.341382369863023,
 'samyra me deu batata frita com cheddar e bacon, e evelyn me deu snickers, amo minhas amigas': -46.13376843222214,
 'crlh acabei de descobrir q tem sorvete de snickers eu preciso pra ontem': -31.47816086843216,
 'yenexiel ninguém compra snickers pra mim quando tô triste': -18.559824881661484,
 'se tu roubar uma birosca dentro da favela os cara da firma de dá um sapeca e tu leva tiro nas mãos o dimenor chicoteado deu sorte mas ainda assim saiu caro esse snickers 😂 https //t co/pirvh7zgb8': -56.06720550470424,
 'rt thaii_xx sério, o anúncio da snickers no yt tá mt chato, eles fazem o bgl pra vender e na real todo mundo vai é ficar c raiva do choco…': -72.2972640101915,
 'rt amandinhavr eu esperando a propaganda insuportável de snickers acabar de passar no vídeo https //t co/d0z

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**